In [1]:
import os
import sys 
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K

from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

import plotly
import plotly.graph_objects as go
import plotly.express as px
import kaleido ##pip install -U kaleido ##to save a plotly fig
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score,cross_val_predict

In [2]:
print(f"Tensor Flow Version: {tf.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.8.0
GPU is available


In [3]:
df1 = pd.read_csv('Stable_Data_CSV.csv')
df2 = pd.read_csv('6wUsingBL.csv')
df_stable = df2[df2.set_index(['participant_id']).index.isin(df1.set_index(['participant_id']).index)]
df_stable_US = df_stable.loc[(df_stable['arm'] == 1)]
df_stable_WN = df_stable.loc[(df_stable['arm'] == 2)]
print(len(df_stable_US))
print(len(df_stable_WN))
df_stable_US=(df_stable_US[['c_3','sl_1','a_2','q_1','tfi_total.1','tfi_total.2','tfi_total.3']])
df_stable_WN=(df_stable_WN[['sl_2','r_1','e_1','tfi_total.1','tfi_total.2','tfi_total.3']])

31
30


In [4]:
def data_prep(df):
    sc_X = StandardScaler()
    sc_y = StandardScaler()
    
    X=df.drop(['tfi_total.2','tfi_total.3'],axis=1)
    y=df[['tfi_total.2']]
    
    #scale x
    x_scaler=sc_X.fit(X)
    X=x_scaler.transform(X)
    #scale y
    y_scaler=sc_y.fit(y)
    y=y_scaler.transform(y)
    
    """
    Reshape rule:
    tensor of shape (batch size, sequence length, features), 
    where sequence length is the number of time steps and features is each input timeseries.
    """
    X = X.reshape((X.shape[0], X.shape[1], 1))
    print(X.shape,y.shape)
    
    return X,y,x_scaler,y_scaler

In [5]:

"""
Include residual connections, layer normalization, and dropout.
The resulting layer can be stacked multiple times.
The projection layers are implemented through `keras.layers.Conv1D`.
"""

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Dense(ff_dim, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(units=inputs.shape[-1])(x) 
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res


def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    K.clear_session()
    inputs = keras.Input(shape=input_shape)
    x = inputs
    
    """    
    can stack multiple of the transformer_encoder blocks and 
    can also proceed to add the final Multi-Layer Perceptron regression head.
    """
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    
    """
    a pooling layer is used to to reduce the output tensor of the TransformerEncoder 
    part of our model down to a vector of features for each data point in the current batch.
    """
    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x) 
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x)
    return keras.Model(inputs, outputs)


In [8]:
## Train and evaluate


def prediction(x_train,y_train,y_scaler):
    input_shape = x_train.shape[1:]

    model = build_model(
        input_shape,
        head_size=5, # Embedding size for each token #key_dim
        num_heads=4, # Number of attention heads
        ff_dim=5, # Hidden layer size in feed forward network inside transformer
        num_transformer_blocks=4,
        mlp_units=[128],
        mlp_dropout=0.4,
        dropout=0.25,
    )

    def error_in_tfi(y_true,y_pred): 
        y=y_true.numpy()
        yhat=y_pred.numpy()
        y=np.reshape(y, (1,-1))
        yhat=np.reshape(yhat, (1,-1))
        y=y_scaler.inverse_transform(y)
        yhat=y_scaler.inverse_transform(yhat)
        y=tf.convert_to_tensor(y, dtype=tf.float32)
        yhat=tf.convert_to_tensor(yhat, dtype=tf.float32)
        return K.mean(abs(y - yhat), axis=-1)  #K.mean(square(y_true - y_pred), axis=-1)


    model.compile(
        loss="mse",
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        #metrics=[keras.metrics.MeanAbsoluteError()],
        run_eagerly=True,
        metrics=[error_in_tfi],
    )
    model.summary()

    callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

    history=model.fit(
        x_train,
        y_train,
        validation_split=0.2,
        epochs=500,
        batch_size=4,
        callbacks=callbacks,
    )
    
    return model,history
    #model.evaluate(x_test, y_test, verbose=1)


In [9]:
error_scores_US=[]
error_scores_WN=[]
kfold = KFold(n_splits=5)

def cross_val(df,error_scores):
    X,y,x_scaler,y_scaler=data_prep(df)
    
    for train_index, test_index in kfold.split(X):
        print("TRAIN:", train_index, "TEST:", test_index)
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
        model,history=prediction(x_train,y_train,y_scaler)
        val_mse,val_mae=model.evaluate(x_test,y_test) #evaluating using unseen data
        error_scores.append(val_mae)
    return history,x_scaler,y_scaler,error_scores

US_history,US_x_scaler,US_y_scaler,US_error_scores=cross_val(df_stable_US,error_scores_US)
WN_history,WN_x_scaler,WN_y_scaler,WN_error_scores=cross_val(df_stable_WN,error_scores_WN)

(31, 5, 1) (31, 1)
TRAIN: [ 7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30] TEST: [0 1 2 3 4 5 6]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5, 1)]       0           []                               
                                                                                                  
 multi_head_attention (MultiHea  (None, 5, 1)        141         ['input_1[0][0]',                
 dAttention)                                                      'input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 5, 1)         0           ['multi_head_attention[0][0]']   
                                                                       

 tf.__operators__.add_6 (TFOpLa  (None, 5, 1)        0           ['layer_normalization_6[0][0]',  
 mbda)                                                            'tf.__operators__.add_5[0][0]'] 
                                                                                                  
 dense_6 (Dense)                (None, 5, 5)         10          ['tf.__operators__.add_6[0][0]'] 
                                                                                                  
 dropout_7 (Dropout)            (None, 5, 5)         0           ['dense_6[0][0]']                
                                                                                                  
 dense_7 (Dense)                (None, 5, 1)         6           ['dropout_7[0][0]']              
                                                                                                  
 layer_normalization_7 (LayerNo  (None, 5, 1)        2           ['dense_7[0][0]']                
 rmalizati

 dense_3 (Dense)                (None, 5, 1)         6           ['dropout_3[0][0]']              
                                                                                                  
 layer_normalization_3 (LayerNo  (None, 5, 1)        2           ['dense_3[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 tf.__operators__.add_3 (TFOpLa  (None, 5, 1)        0           ['layer_normalization_3[0][0]',  
 mbda)                                                            'tf.__operators__.add_2[0][0]'] 
                                                                                                  
 multi_head_attention_2 (MultiH  (None, 5, 1)        141         ['tf.__operators__.add_3[0][0]', 
 eadAttention)                                                    'tf.__operators__.add_3[0][0]'] 
          

5/5 [==============================] - 1s 132ms/step - loss: 0.3821 - error_in_tfi: 9.5019 - val_loss: 0.4383 - val_error_in_tfi: 7.9485
Epoch 66/500
5/5 [==============================] - 1s 134ms/step - loss: 0.3239 - error_in_tfi: 8.2129 - val_loss: 0.4349 - val_error_in_tfi: 7.9326
Epoch 67/500
5/5 [==============================] - 1s 140ms/step - loss: 0.3642 - error_in_tfi: 9.2069 - val_loss: 0.4314 - val_error_in_tfi: 7.8805
Epoch 68/500
5/5 [==============================] - 1s 137ms/step - loss: 0.3183 - error_in_tfi: 8.0865 - val_loss: 0.4269 - val_error_in_tfi: 7.7093
Epoch 69/500
5/5 [==============================] - 1s 140ms/step - loss: 0.3737 - error_in_tfi: 9.1869 - val_loss: 0.4236 - val_error_in_tfi: 7.5332
Epoch 70/500
5/5 [==============================] - 1s 153ms/step - loss: 0.3251 - error_in_tfi: 8.9024 - val_loss: 0.4219 - val_error_in_tfi: 7.4968
Epoch 71/500
5/5 [==============================] - 1s 145ms/step - loss: 0.3678 - error_in_tfi: 8.9288 - val_los

                                                                                                  
 dropout_5 (Dropout)            (None, 5, 5)         0           ['dense_4[0][0]']                
                                                                                                  
 dense_5 (Dense)                (None, 5, 1)         6           ['dropout_5[0][0]']              
                                                                                                  
 layer_normalization_5 (LayerNo  (None, 5, 1)        2           ['dense_5[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 tf.__operators__.add_5 (TFOpLa  (None, 5, 1)        0           ['layer_normalization_5[0][0]',  
 mbda)                                                            'tf.__operators__.add_4[0][0]'] 
          

...

                                                                                                  
 dropout_8 (Dropout)            (None, 128)          0           ['dense_8[0][0]']                
                                                                                                  
 dense_9 (Dense)                (None, 1)            129         ['dropout_8[0][0]']              
                                                                                                  
Total params: 1,413
Trainable params: 1,413
Non-trainable params: 0
__________________________________________________________________________________________________
Epoch 1/500
5/5 [==============================] - 1s 154ms/step - loss: 0.8729 - error_in_tfi: 17.2177 - val_loss: 1.0897 - val_error_in_tfi: 18.4656
Epoch 2/500
5/5 [==============================] - 1s 130ms/step - loss: 0.8712 - error_in_tfi: 17.3178 - val_loss: 1.1238 - val_error_in_tfi: 18.8127
Epoch 3/500
5/5 [===================

Epoch 50/500
5/5 [==============================] - 1s 137ms/step - loss: 0.3757 - error_in_tfi: 10.2420 - val_loss: 0.3449 - val_error_in_tfi: 9.5255
Epoch 51/500
5/5 [==============================] - 1s 136ms/step - loss: 0.4077 - error_in_tfi: 10.8421 - val_loss: 0.3316 - val_error_in_tfi: 9.2119
Epoch 52/500
5/5 [==============================] - 1s 136ms/step - loss: 0.4033 - error_in_tfi: 11.5290 - val_loss: 0.3252 - val_error_in_tfi: 8.9567
Epoch 53/500
5/5 [==============================] - 1s 130ms/step - loss: 0.4055 - error_in_tfi: 11.3468 - val_loss: 0.3360 - val_error_in_tfi: 8.9523
Epoch 54/500
5/5 [==============================] - 1s 134ms/step - loss: 0.3588 - error_in_tfi: 9.8299 - val_loss: 0.3363 - val_error_in_tfi: 8.8458
Epoch 55/500
5/5 [==============================] - 1s 135ms/step - loss: 0.4753 - error_in_tfi: 11.5020 - val_loss: 0.3254 - val_error_in_tfi: 8.6768
Epoch 56/500
5/5 [==============================] - 1s 141ms/step - loss: 0.3811 - error_in_tfi

 rmalization)                                                                                     
                                                                                                  
 tf.__operators__.add_3 (TFOpLa  (None, 4, 1)        0           ['layer_normalization_3[0][0]',  
 mbda)                                                            'tf.__operators__.add_2[0][0]'] 
                                                                                                  
 multi_head_attention_2 (MultiH  (None, 4, 1)        141         ['tf.__operators__.add_3[0][0]', 
 eadAttention)                                                    'tf.__operators__.add_3[0][0]'] 
                                                                                                  
 dropout_4 (Dropout)            (None, 4, 1)         0           ['multi_head_attention_2[0][0]'] 
                                                                                                  
 layer_nor

 da)                                                              'input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 4, 5)         10          ['tf.__operators__.add[0][0]']   
                                                                                                  
 dropout_1 (Dropout)            (None, 4, 5)         0           ['dense[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 4, 1)         6           ['dropout_1[0][0]']              
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 4, 1)        2           ['dense_1[0][0]']                
 rmalization)                                                                                     
          

 global_average_pooling1d (Glob  (None, 4)           0           ['tf.__operators__.add_7[0][0]'] 
 alAveragePooling1D)                                                                              
                                                                                                  
 dense_8 (Dense)                (None, 128)          640         ['global_average_pooling1d[0][0]'
                                                                 ]                                
                                                                                                  
 dropout_8 (Dropout)            (None, 128)          0           ['dense_8[0][0]']                
                                                                                                  
 dense_9 (Dense)                (None, 1)            129         ['dropout_8[0][0]']              
                                                                                                  
Total para

In [12]:
print("US_error")
print("--------")
print(US_error_scores)
US_error=np.mean(US_error_scores)
print("\n")
print(US_error)
print("\n")
print("WN_error")
print("--------")
print("WN_error_scores")
WN_error=np.mean(WN_error_scores)
print("\n")
print(WN_error)

US_error
--------
[13.278341293334961, 12.599403381347656, 20.60761260986328, 13.28161907196045, 14.415968894958496]


14.836589050292968


WN_error
--------
[17.308361053466797, 14.438372611999512, 10.359463691711426, 24.019689559936523, 13.964017868041992]


16.01798095703125
